## 1.9
Each of these two procedures defines a method fro adding two numbers. show how they evaluate with 4 and 5. Are they iterative or recursive?

```scheme
(define (+ a b)
    (if (= a 0)
        b
        (inc (+ (dec a) b))))

```

results:
```scheme
(+ 4 5)
(+ 1 (+ 3 5))
(+ 1 (+ 1 (+ 1 (2 5))))
(+ 1 (+ 1 (+ 1 (+ 1 5))))
(+ 1 (+ 1 (+ 1 (+ 1 (+ 0 5)))))
(+ 1 (+ 1 (+ 1 (+ 1 5))))
(+ 1 (+ 1 (+ 1 6)))
(+ 1 (+ 1 7))
(+ 1 8)
9
```

obviously recursive. the `inc` has to stay on the stack until `(+ (dec a) b)` evaluates (which is recursive, since it's defined in terms of the `+`)

```scheme
(define (+ a b)
    (if (= a 0)
        b
        (+ (dec a) (inc b))))
```
results:
```scheme
(+ 4 5)
(+ (- 1 4) (+ 1 5))
(+ 3 6)
(+ (- 1 3) (+ 1 6))
(+ 2 7)
(+ (- 1 2) (+ 1 7))
(+ 1 8)
(+ (- 1 1) (+ 1 8))
(+ 0 9)
9
done
```

iterative- before we enter the recursive call to `+`, we have to evaluate `(dec a)` and `(inc b)` in the current stack frame

## 1.10

this computes Ackermann's function (wtf is Ackermann's function?)

```scheme
(define (A x y)
    (cond ((= y 0) 0)
        ((= x 0) (* 2 y))
        ((= y 1) 2)
        (else (A (- x 1)
                 (A x (- y 1))))))
```
evaulate the following:

```scheme
(A 1 10)
(A 2 4)
(A 3 3)
```

1.
```scheme
(A 1 10)
(A 0 (A 1 9))
(A 0 (A 0 (A 1 8))
(A 0 (A 0 (A 0 (A 1 7)))
(A 0 (A 0 (A 0 (A 1 (A 1 6))))
...
(A 0 (A 0 (A 0 (A 0 (A 0 A(0 (A 0 (A 0 (A 0 (A 0 1)))))))))
```
I think it's just 2^9?  

In [8]:
(define (pow x y)
        (if (= y 0) 1 (* x (pow x (- y 1)))))

(pow 2 10)

1024

(guess it's 2^10)

2.
```scheme
(A 2 4)
(A 1 (A 2 3))
(A 1 (A 1 (A 2 2)))
(A 1 (A 1 (A 1 (A 2 1))))
(A 1 (A 1 (A 1 2)))
(A 1 (A 1 (A 0 (A 1 1)))
(A 1 (A 1 (A 0 2)))
(A 1 (A 1 4))
(A 1 (A 0 (A 1 3)))
(A 1 (A 0 (A 0 (A 1 2))))
(A 1 (A 0 (A 0 (A 0 (A 1 1))))))
(A 1 (A 0 (A 0 (A 0 2)))))
(A 1 (A 0 (A 0 4))))
(A 1 (A 0 8)))
(A 1 16))
2^16
65536
```
(A 0 n) = 2n  
(A 1 n) = 2^n  
(A 2 n) = ??? 2^2^2^2 n times?

3.
```scheme
(A 3 3)
(A 2 (A 3 2)
(A 2 (A 2 (A 3 1)))
(A 2 (A 2 2)
(A 2 (A 1 (A 2 1)))
```

## 1.11
`f(x)` is defined as follows:  
if `n < 3`, `f(n) = n`  
otherwise `f(n) = f(n-1) + 2*f(n-2) + 3*f(n-3)`

define a recursive function that does this and an iterative one that does the same

In [6]:
(define (fn n)
  (if (< n 3) n
      (+ (fn (- n 1)) (* 2 (fn (- n 2))) (* 3 (fn (- n 3))))))

(fn 5)

25

In order to do the iterative version of this, we're basically tail-calling everything. So, what do we need to tail call?

In every stack frame, there are three places that we're opening new stack frames. those are as follows:
* `(fn (- n 1))`
* `(fn (- n 2))`
* `(fn (- n 3))`

These are 3 things that we want to compute in our current stack frame instead of letting the next one handle it. How do we do this?

in order to compute `fn(n)`, we have to compute `fn(n-1)` and so on, all the way down to `fn(n)` where `n < 3`, which is our base case.

Let's start looking at it from there.

```scheme
(fn 0) = 0
(fn 1) = 1
(fn 2) = 2
(fn 3) = (+ (fn 2) (* 2 (fn 1)) (* 3 (fn 0)))
```

we already have `(fn 2)`, `(fn 1)`, and `(fn 0)`, so we can sub them in

```scheme
(fn 3) = (+ 2 (* 2 1) (* 3 0))
(fn 3) = (+ 2 2 0)
(fn 3) = 4
```

and we can keep going
```scheme
(fn 4) = (+ (fn 3) (* 2 (fn 2)) (* 3 (fn 1)))
         (+ 4 (* 2 2) (* 3 1))
         (+ 4 4 3)
         11
```
eventually we'll hit `(fn n)`

Instead of using stack frames to calculate state, let's create variables. we'll have `a, b, c`

```scheme
(define (iterate a b c)
        ())
```

let's not worry about how many times to do it yet, or what the base case is. how do we describe behavior in the middle of the loop? Let's imagine that `a b c` correspond to `fn(n-1), fn(n-2), fn(n-3)` respectively. we'll take it as a given that we'll be able to get those values.

What happens in the next iteration?

Well, the value of `f(n)` at the current `n` becomes the value of `f(n-1)` if we're moving "up" a level. i.e. `f(4)` up to `f(5)`, naturally `f(4)` is now the `f(n-1)` in the context of `f(5)`. Same thing happens to `f(n-1)`, which becomes `f(n-2)` when our `n` increases by `1`. everything "shifts" over. we don't use `f(n-3)` at all, so `c` simply ceases to exist.

```scheme
(define (iterate a b c)
        (iterate _ a b))
```

so, `a` becomes `b`, `b` becomes `c`. What's our new `a` then? if `a` is `f(n-1)`, we need to find `f(n)` at our current `n` to shift "right". Well, the initial function gives it to us. `f(n-1) + 2*f(n-2) + 3*f(n-3)`. Since we're using `a b c` as stand-ins for `f(n-1), f(n-2), f(n-3)`, we can simply substitute the variables for the function invocations. we get `a + 2*b + 3*c`. Let's plug that in here:

```scheme
(define (iterate a b c)
        (iterate (+ a (* 2 b) (* 3 c)) a b))
```

nice. Now let's think about where to start and stop. we can't start at the end, the way that the recursive solution does. the recursive function piles on stack frames until we hit our base case of `n < 3`. Instead we have to pile "up" from our base case up to `n`. we know that `f(0) = 0`, `f(1) = 1`, and `f(2) = 2`, so let's take that as a starting point.

```scheme
(define (iterate a b c)
        (iterate (+ a (* 2 b) (* 3 c)) a b)))
        
(iterate 2 1 0)
```

calling `iterate` with `2 1 0` will start at `f(3)`, since we're providing it values equivalent to `f(2)`, `f(1)`, and `f(0)`. it will compute `f(3)`, then `f(4)`, then `f(5)` endlessly. Let's start at `n`, and count down until `n < 3`

```scheme
(define (iterate a b c n)
        (if (< n 3) a
        (iterate (+ a (* 2 b) (* 3 c)) a b (- n 1))))
        
(iterate 2 1 0 5)
```
here's a test run:

In [42]:
(define (iterate a b c n)
        (if (< n 3) a
        (iterate (+ a (* 2 b) (* 3 c)) a b (- n 1))))
        
(iterate 2 1 0 5)

25

our initial `a b c` arguments will never change, so let's not fuss about those. we can put them behind another layer of abstraction, and add a conditional that will handle `x < 3`, since our `iterate` function was built to handle cases where `x > 3`. Here's how it looks:

In [43]:
(define (f x)
  (define (iterate a b c n)
        (if (< n 3) a
        (iterate (+ a (* 2 b) (* 3 c)) a b (- n 1))))
  (if (< x 3 x)
      (iterate 2 1 0 x)))

f

## 1.12 - pascal's triangle

write a function to recursively generate elements of pascal's triangle

In [8]:
(define (pascal level item) 
  (if (and (= level 0) (= item 0))
       1
       (if (or (< item 0) (> item level))
           0
           (+ (pascal (- level 1) item) (pascal (- level 1) (- item 1))))))

(pascal 3 2)

3

## 1.13 - phi^n / sqrt(5)

prove that fib(n) is the closest integer to phi^n/sqrt(5) where phi = (1 + sqrt(5))/2, fib(n) = (phi^n - psi^n)/sqrt(5), and psi = (1-sqrt(5))/2

uhhh go check out [this](https://codology.net/post/sicp-solution-exercise-1-13/)


## 1.14

draw the tree for a recursive computation of the coin change problem (given n amount and an array of coin values, how many ways can we make change?)

I'm not gonna draw this one, but here's how we think of the tree.

each node has x children, where x is the number of denominations of coins remaining in the array. if we think of "keeping" coins as staying on the "right", the further "left" we go, the fewer coins remain in the array.

i.e.

`coin-change(100, [25, 10, 5, 1])`

will have children that look something like this:  

```
coin-change(100, [])
coin-change(100, [25]) 
coin-change(100, [25, 10]) 
coin-change(100, [25, 10, 5]) 
coin-change( 99, [25, 10, 5, 1])
```

I think that, as amounts are added, the order of change is something like ^(n of coins)

## 1.15 - TODO

## 1.16

write a fast exponentiation function that is tail-recursive

recursive version:

```scheme
(define (fast-expt b n)
  (cond ((= n 0) 1)
        ((even? n) (square (fast-expt b (/ n 2))))
        (else (* b (fast-expt b (- n 1))))))

(define (even? n)
        (= (modulo n 2) 0))
```

In [ ]:
(define (exp-iter a b x)
        (cond ((= b 0) x)
              (else (fast-expt a (-b 1) (* a a)))))

(define (fast-exp (a b x)
                  (cond ((= 0 a) x)
                        (even? a) (fast-exp (/ a 2) (* b b) x)
                        (else (fast-exp (- a 1) b (* b x))))
                  
(fast-exp a b 1)

I didn't write that one, but we can at least break it down.

every time we're doing exponentiation, we're figuring out how to get to the next step. Our third parameter, the `x` is used to store a running total of what our current value is at the current step, similar to a local variable.

if I were to write it in python, it'd look something like this:

```python
def fast-exp(a, b):
    result = 1
    while b > 0:
        if b%2 == 0:
            b = b/2
            result = result * result
        else:
            b = b - 1
            result = result * b
    return result
```

if we translate it into a specifically tail recursive form, we get the following:

```python
def fast-exp(a, b):
    def iter(a, b, count):
        if b < 1:
            return count
        elif b%2 == 0:
            return iter(a, b/2, count*count)
        else:
            return iter(a, b-1, count*b)
    return iter(a, b, 1)
```

